# Introduction to Machine Learning: Task 1b

__Author__: Jannick Sicher

### Initial Configurations and Packages

In [68]:
# Load packages
import numpy
import matplotlib.pyplot as plot
import pandas
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [69]:
# Load data
df = pd.read_csv("train.csv")
df.head()

,Id,y,x1,x2,x3,x4,x5
0,0,-5.522114,1.764052,0.400157,0.978738,2.240893,1.867558
1,1,-21.789980,-0.977278,0.950088,-0.151357,-0.103219,0.410599
2,2,-7.911497,0.144044,1.454274,0.761038,0.121675,0.443863
3,3,-3.698062,0.333674,1.494079,-0.205158,0.313068,-0.854096
4,4,-16.001833,-2.552990,0.653619,0.864436,-0.742165,2.269755


In [70]:
# Define Target Variable
y = df["y"] # define the target variable (dependent variable) as y
Xs = df.drop(['Id','y'], axis = 1)
# Train and Test Split
X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(800, 5) (800,)
(200, 5) (200,)


### Linear Regression

In [71]:
#initialisation of linear regression 
linear_model = LinearRegression()

#inserting the train data in model for training purpose 
linear_model.fit(X_train,y_train)
#now model trained with this data

#now we will do prediction for the data 
y_pred = linear_model.predict(X_test)

#test our result how accurate our model is 
error = y_test-y_pred

#e= np.sum(error)
#abserror = np.absolute(error)
#sqrarr= np.square(error)
#addall= np.sum(sqrarr)
print('Coefficients: \n', linear_model.coef_)
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))

Coefficients: 
 [ 2.61821714 -1.339253   -1.83134595  0.36994903 -0.58503927]
Mean squared error: 104.49


### Quadratic Transformation

In [72]:
Xq = df[['x1', 'x2', 'x3', 'x4', 'x5']]**2
Xq.head()

,x1,x2,x3,x4,x5
0,3.111881,0.160126,0.957928,5.021602,3.487773
1,0.955072,0.902668,0.022909,0.010654,0.168591
2,0.020749,2.114911,0.579178,0.014805,0.197015
3,0.111339,2.232272,0.042090,0.098011,0.729480
4,6.517757,0.427217,0.747250,0.550809,5.151786


In [73]:
# Train and Test Split
X_train, X_test, y_train, y_test = train_test_split(Xq, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(800, 5) (800,)
(200, 5) (200,)


In [74]:
#initialisation of linear regression 
linear_model = LinearRegression()

#inserting the train data in model for training purpose 
linear_model.fit(X_train,y_train)
#now model trained with this data

#now we will do prediction for the data 
y_pred = linear_model.predict(X_test)

#test our result how accurate our model is 
error = y_test-y_pred

#e= np.sum(error)
#abserror = np.absolute(error)
#sqrarr= np.square(error)
#addall= np.sum(sqrarr)
print('Coefficients: \n', linear_model.coef_)
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))

Coefficients: 
 [-0.64354644 -0.06751144 -1.36921758 -0.10642783  0.47094992]
Mean squared error: 108.96


### Exponential Transformation

In [75]:
Xe = np.exp(df[['x1', 'x2', 'x3', 'x4', 'x5']])
Xe.head()

,x1,x2,x3,x4,x5
0,5.836039,1.492059,2.661096,9.401725,6.472471
1,0.376334,2.585938,0.859541,0.901930,1.507720
2,1.154934,4.281372,2.140496,1.129387,1.558717
3,1.396088,4.455232,0.814518,1.367614,0.425668
4,0.077849,1.922485,2.373667,0.476082,9.677026


In [76]:
# Train and Test Split
X_train, X_test, y_train, y_test = train_test_split(Xe, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(800, 5) (800,)
(200, 5) (200,)


In [77]:
#initialisation of linear regression 
linear_model = LinearRegression()

#inserting the train data in model for training purpose 
linear_model.fit(X_train,y_train)
#now model trained with this data

#now we will do prediction for the data 
y_pred = linear_model.predict(X_test)

#test our result how accurate our model is 
error = y_test-y_pred

#e= np.sum(error)
#abserror = np.absolute(error)
#sqrarr= np.square(error)
#addall= np.sum(sqrarr)
print('Coefficients: \n', linear_model.coef_)
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))

Coefficients: 
 [ 0.85539307 -0.63452424 -1.20693906  0.05934347  0.14956245]
Mean squared error: 105.51


### Cosine Transformation

In [78]:
Xc = np.cos(df[['x1', 'x2', 'x3', 'x4', 'x5']])
Xc.head()

,x1,x2,x3,x4,x5
0,-0.192055,0.921000,0.558070,-0.621062,-0.292425
1,0.559281,0.581611,0.988567,0.994678,0.916882
2,0.989644,0.116259,0.724121,0.992607,0.903099
3,0.944845,0.076642,0.979029,0.951393,0.656901
4,-0.831717,0.793889,0.649069,0.737007,-0.643421


In [79]:
# Train and Test Split
X_train, X_test, y_train, y_test = train_test_split(Xc, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(800, 5) (800,)
(200, 5) (200,)


In [80]:
#initialisation of linear regression 
linear_model = LinearRegression()

#inserting the train data in model for training purpose 
linear_model.fit(X_train,y_train)
#now model trained with this data

#now we will do prediction for the data 
y_pred = linear_model.predict(X_test)

#test our result how accurate our model is 
error = y_test-y_pred

#e= np.sum(error)
#abserror = np.absolute(error)
#sqrarr= np.square(error)
#addall= np.sum(sqrarr)
print('Coefficients: \n', linear_model.coef_)
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))

Coefficients: 
 [ 1.77304264 -0.36827784  3.54598953 -0.09951397 -1.5556242 ]
Mean squared error: 118.58


### Constant Transformation

In [81]:
Xconst = np.ones(1000)
df['Xconst']= Xconst
Xconst = Xconst.reshape(-1,1)
df.head()

,Id,y,x1,x2,x3,x4,x5,Xconst
0,0,-5.522114,1.764052,0.400157,0.978738,2.240893,1.867558,1.0
1,1,-21.789980,-0.977278,0.950088,-0.151357,-0.103219,0.410599,1.0
2,2,-7.911497,0.144044,1.454274,0.761038,0.121675,0.443863,1.0
3,3,-3.698062,0.333674,1.494079,-0.205158,0.313068,-0.854096,1.0
4,4,-16.001833,-2.552990,0.653619,0.864436,-0.742165,2.269755,1.0


In [82]:
# Train and Test Split
X_train, X_test, y_train, y_test = train_test_split(Xconst, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(800, 1) (800,)
(200, 1) (200,)


In [83]:
#initialisation of linear regression 
linear_model = LinearRegression()

#inserting the train data in model for training purpose 
linear_model.fit(X_train,y_train)
#now model trained with this data

#now we will do prediction for the data 
y_pred = linear_model.predict(X_test)

#test our result how accurate our model is 
error = y_test-y_pred

#e= np.sum(error)
#abserror = np.absolute(error)
#sqrarr= np.square(error)
#addall= np.sum(sqrarr)
print('Coefficients: \n', linear_model.coef_)
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))

Coefficients: 
 [0.]
Mean squared error: 92.45
